In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
analyser = SentimentIntensityAnalyzer()

In [21]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    del score['compound']
    return max(score.items(), key=operator.itemgetter(1))[0]

In [9]:
import os
import re
df = pd.DataFrame()
_id = 1
for filename in os.listdir('./speeches'):
    if filename == '.DS_Store':
        continue
    for speech in os.listdir('./speeches/' + filename):
        temp = open('./speeches/' + filename + '/' + speech, 'r', encoding='utf-8').readlines()
        obj = {}
        obj['doc_id'] = _id
        date = re.findall('"([^"]*)"', temp[1])
        obj['date'] = date[0] if len(date) > 0 else None
        obj['pres'] = filename
        obj['title'] = re.findall('"([^"]*)"', temp[0])[0]
        obj['speech']= "".join(temp[2:])
        obj['sentiment'] = ""
    
        obj = pd.DataFrame(obj, index=[0])
        df = df.append(obj, ignore_index=True)
        _id += 1
df = df.set_index("doc_id")
df.head()

,date,pres,title,speech,sentiment
doc_id,,,,,
1,"August 10, 1927",coolidge,Address at the Opening of Work on Mount Rushmo...,We have come here to dedicate a cornerstone th...,
2,"December 8, 1925",coolidge,Third Annual Message,Members of the Congress: In meeting the consti...,
3,"December 6, 1923",coolidge,First Annual Message,Since the close of the last Congress the Natio...,
4,"October 20, 1925",coolidge,Message Regarding Relationship of Church and S...,"Mr. Moderator, Members Of The Council:\nIt is ...",
5,"March 4, 1925",coolidge,Inaugural Address,\nMy Countrymen:\n\nNo one can contemplate cur...,


In [10]:
data = df.speech.values.tolist()
data = [re.sub('\n+', ' ', sent) for sent in data]
data = [re.sub(r"\<.*\>"," ",sent) for sent in data]

In [27]:
import operator
for index, row in df.iterrows():
    row['speech'] = re.sub('\n+',' ',row['speech'])
    row['speech'] = re.sub(r'\<.*\>',' ',row['speech'])
    row['sentiment'] = sentiment_analyzer_scores(row['speech'])


KeyboardInterrupt: 

In [32]:
pres_list = "adams arthur bharrison buchanan bush carter cleveland cleveland2 clinton coolidge eisenhower fdroosevelt fillmore ford garfield grant gwbush harding harrison hayes hoover jackson jefferson johnson jqadams kennedy lbjohnson lincoln madison mckinley monroe nixon obama pierce polk reagan roosevelt taft taylor truman trump tyler vanburen washington wilson".split()
pres_sentiments = {}
for pres in pres_list:
    pres_sentiments[pres] = 0

for pres in pres_list:
    df_pres = df[df['pres']==pres]
    num_pos = len(df_pres[df_pres['sentiment']=='pos'])
    pres_sentiments[pres] = num_pos / len(df_pres)
pres_sentiments

{'adams': 0.0,
 'arthur': 0.0,
 'bharrison': 0.0,
 'buchanan': 0.0,
 'bush': 0.0,
 'carter': 0.0,
 'cleveland': 0.0,
 'cleveland2': 0.0,
 'clinton': 0.02564102564102564,
 'coolidge': 0.0,
 'eisenhower': 0.0,
 'fdroosevelt': 0.0,
 'fillmore': 0.0,
 'ford': 0.0,
 'garfield': 0.0,
 'grant': 0.0,
 'gwbush': 0.0,
 'harding': 0.0,
 'harrison': 0.0,
 'hayes': 0.0,
 'hoover': 0.0,
 'jackson': 0.0,
 'jefferson': 0.0,
 'johnson': 0.0,
 'jqadams': 0.0,
 'kennedy': 0.0,
 'lbjohnson': 0.0,
 'lincoln': 0.0,
 'madison': 0.0,
 'mckinley': 0.0,
 'monroe': 0.0,
 'nixon': 0.0,
 'obama': 0.22916666666666666,
 'pierce': 0.0,
 'polk': 0.0,
 'reagan': 0.01694915254237288,
 'roosevelt': 0.0,
 'taft': 0.0,
 'taylor': 0.0,
 'truman': 0.0,
 'trump': 0.5121951219512195,
 'tyler': 0.0,
 'vanburen': 0.0,
 'washington': 0.0,
 'wilson': 0.0}

In [33]:
df[df['pres']=='trump']

,date,pres,title,speech,sentiment
doc_id,,,,,
771,2016-10-10,trump,"Donald Trump, Republican Presidential Candidat...","Thank you. Thank you very much. Thank you, e...",pos
772,2016-07-26,trump,"Donald Trump, Republican Presidential Candidat...",,neg
773,2016-10-17,trump,"Donald Trump, Republican Presidential Candidat...",Thank you. Thank you. Thank you very much. G...,pos
774,2016-09-07,trump,"Donald Trump, Republican Presidential Candidat...",Thank you very much and God bless you. Thank...,pos
775,2016-10-10,trump,"Donald Trump, Republican Presidential Candidat...",We are going to vote on November 8th to make...,pos
776,2016-09-08,trump,"Donald Trump, Republican Presidential Candidat...",See you in four years suckers. But the failu...,neu
777,2016-07-25,trump,"Donald Trump, Republican presidential candidat...","Thank you, everybody. Thank you.",pos
778,2016-09-09,trump,"Donald Trump, Republican Presidential Candidat...",It's time to stop quibbling over the smalles...,neu
779,2016-10-28,trump,"Donald Trump, Republican Presidential Candidat...","Thank you. Thank you, New Hampshire.",pos
